In [9]:

import numpy as np
import os
from pprint import pprint
import pandas as pd
import pulp
import random
# import storage
from storage.warehouse_input_parser import WarehouseInputParser
from storage.containers import Bin, Tote, BulkContainer
from storage.item import Item
import time





    
def get_units(n_units=3):
    # create Item objects
    # if N_UNITS is None:
    #     N_UNITS=
    container_types = (Bin, Tote, BulkContainer)
    containers = []
    items = []


    for i in range(n_units):
        container = np.random.choice(container_types, p=[0.8,0.1,0.1])()
        container.priority = np.random.choice(('A','B','C'), p=[0.1,0.8,0.1])
        
        # btw 1 and 6 items per container
        n_items = random.randint(1,6)
        for i in range(n_items):
            item = Item(priority = container.priority)
            items.append(item)
            c = container
            c.d = c.dimensions['outer']
            c.l, c.w, c.h = c.d['length'], c.d['width'], c.d['height']
            c.add_item_to_container(item)
            

        containers.append(c)

        
    units = [ 
            {'ID':c.uuid.hex,
             'Name': c.name,
             'Full Name': c.full_name,
             'Length':c.dimensions['outer']['length'], 
             'Width':c.dimensions['outer']['width'], 
             'Height':c.dimensions['outer']['height'],
             'Volume': c.l*c.w*c.h,
             'Priority':c.priority,
             'Weight': min(c.weight, 2000)} for c in containers ]
    
    # for unit in units:
    #     unit['Volume'] = unit['Length']*unit['Width']*unit['Height']


    return items, containers, units




# return the unit by ID
def get_unit_by_id(unitID):
    for unit in units:
        if unitID == unit['ID']:
            return unit

# return the item by ID
def get_item_by_id(unitID):
    for item in items:
        if unitID == item.uuid:
            return item

def get_container_by_id(containerID):
    for container in containers:
        if containerID == container.uuid.hex:
            return container
    

def get_warehouse(warehouse_path='warehouse-layout-input.xlsx'):
    
    warehouse = WarehouseInputParser(warehouse_path).process_warehouse_input()
    all_shelf_ids = list(warehouse['ShelfID'])
    shelves = warehouse.to_dict('records')
    for shelf in shelves:
        shelf['Volume'] = shelf.get('ShelfLength')*shelf.get('ShelfWidth')*shelf.get('ShelfHeight')
    shelf_count = len(all_shelf_ids)
    
    return warehouse, all_shelf_ids, shelves, shelf_count

# warehouse, all_shelf_ids, shelves, shelf_count = get_warehouse()

def get_shelf_by_id(ShelfID):
    for shelf in shelves:
        if shelf['ShelfID'] == ShelfID:
            return shelf
        

# items, containers, units = get_units(n_units=N_UNITS)



def optimize_function(  x=dict(), # old soln x
                        # y=dict(), # old soln y
                        n_units=3, # number of units to add
                        old_units=[],
                        new_units=[],
                        shelves=None,
                        all_shelf_ids=None):


    # store solution for x
    solutionX = dict()
    for k, v in x.items():
        solutionX[k] = v.value()

    # # store solution for y
    # solutionY = dict()
    # for k, v in y.items():
    #     solutionY[k] = v.value()

    units = old_units + new_units

    print(f'There are now {len(units)} units.')




    '''
    Variables
    '''

    shelf_loc = [(u.get('ID'), shelfID) for u in units for shelfID in all_shelf_ids]


    # # x is options for item + bin combinations
    x = pulp.LpVariable.dicts(
        'OnShelf', shelf_loc,
        lowBound = 0, # item is not on the shelf
        upBound = 1, # item is on the shelf
        cat = 'Integer' # solver uses integers, i think?
    )

    # z is a dummy variable for shelf waste; minimizing it then using it in a constraint acts like taking the max would
    z = pulp.LpVariable.dicts(
        'ShelfWaste', [(unit.get('ID'), shelf.get('ShelfID')) for unit in units for shelf in shelves],
        0,
        cat = 'Integer'
    )

    # add bounds unique to each shelf
    for i in z:
        for shelf in shelves:
            if shelf.get('ShelfID') == i:
                z[i].bounds(0, shelf.get('Volume'))


    # initialize problem
    prob = pulp.LpProblem('Shelf_Stocking', pulp.LpMinimize)

    '''
    Objective Function
    '''
    prob += pulp.lpSum([z[(unit.get('ID'), shelf.get('ShelfID'))] for unit in units for shelf in shelves]), 'Objective: Minimize Excess Shelf Space'

    '''
    constraints
    '''

    # enforce old solution
    for k,v in solutionX.items():
        for i in range(shelf_count):
            if int(v) == 1:
                try:
                    prob += x[k] == 1.0, f'Enforce old solution {k}'
                except:
                    pass

    for unit in units:
        prob += pulp.lpSum([x[(unit.get('ID'), shelfID)] for shelfID in all_shelf_ids]) == 1, 'Item {} can only be on one shelf'.format(unit.get('ID'))


    # # sum of item widths should not exceed shelf width ("length")
    for i, shelf in enumerate(shelves):
        prob += pulp.lpSum([unit.get('Length') * x[(unit.get('ID'), shelf['ShelfID'])] for unit in units]) <= shelf['ShelfLength'], 'Sum of widths cannot exceed shelf {} width'.format(i)


        # the sum of item weights should not exceed shelf capacity
        prob += pulp.lpSum([unit.get('Weight') * x[(unit.get('ID'), shelf['ShelfID'])] for unit in units]) <= shelf['WeightCapacity'], 'Sum of weights cannot exceed shelf {} weight cap'.format(i)


        for unit in units:
            # minimize shelf waste
            prob += ( shelf.get('Volume') - unit.get('Volume')) * x[(unit.get('ID'), shelf.get('ShelfID'))] <= z[unit.get('ID'), shelf.get('ShelfID')], 'Minimize wasted space created by inserting item item {} onto shelf {}'.format(unit.get('ID'), shelf.get('ShelfID'))


    # the difference in priority btw item and shelf should be 0
    prob+= pulp.lpSum([abs(ord(unit.get('Priority')) - ord(shelf.get('ShelfPriority'))) * x[(unit.get('ID'), shelf['ShelfID'])]\
                        for unit in units for shelf in shelves ]) == 0, 'All items are matched to priority'
    
    
    # set initial values
    for k, v in solutionX.items():
        if int(v) == 1:
            # print(k,'-->',v)
            x[k].setInitialValue(v)
    
    

    # '''
    # Solve
    # '''
    start_time = time.time()
    solver = pulp.PULP_CBC_CMD(msg=True)
    prob.solve(solver)
    print('Solved in {:.3f} seconds.'.format(time.time() - start_time))

    return x, z, items, containers, units


def get_stocked_shelves(x):
    stocked_shelves = {}
    for item in x.keys():
        if x[item].value()==1:
            itemNum = item[0]
            shelfNum = item[1]
            if shelfNum in stocked_shelves:
                stocked_shelves[shelfNum].append(itemNum)
            else:
                stocked_shelves[shelfNum] = [itemNum]
    return stocked_shelves

# pprint(get_stocked_shelves(x))



def detail_report(stocked_shelves, shelves, units):
 
    for shelfID, unitIDs in stocked_shelves.items():
        print(f'\nShelf {shelfID}:')
        shelfdata = get_shelf_by_id(shelfID, shelves)
        print(f"\t{shelfdata['ShelfType']}\tpriotiry: {shelfdata['ShelfPriority']}\tcontains {unitIDs}\n")

 
        for unitID in unitIDs:
            unit = get_unit_by_id(unitID, units)
            pprint(unit)


def put_away(units=[],
             x=dict(),
             warehouse_path='warehouse-layout-input.xlsx'):

    warehouse, all_shelf_ids, shelves, shelf_count = get_warehouse(warehouse_path=warehouse_path)

    xold = x
    x, z, items, containers, units = optimize_function(x=xold,
                                                       new_units=units,
                                                      all_shelf_ids=all_shelf_ids,
                                                      shelves=shelves)

    return [get_stocked_shelves(xold),get_stocked_shelves(x)]




In [10]:
units

[{'ID': '3d3a6b60d39f4321808a8b15b9cdeb79',
  'Name': 'bin',
  'Full Name': 'modular nesting container',
  'Length': 11,
  'Width': 23.5,
  'Height': 9,
  'Volume': 2326.5,
  'Priority': 'B',
  'Weight': 146}]

In [11]:
items, containers, units = get_units(1)

In [12]:
put_away(units=units)

There are now 1 units.
Solved in 1.213 seconds.


[{}, {(86, 23, 'C'): ['3462b2b607c044efbfb687fd0cc244ab']}]